In [ ]:

!apt-get update && apt-get install -y python3-opencv
!pip install tensorflow==2.12.0
# !pip install keras-cv tensorflow --upgrade
!pip install opencv-python
!pip install scikit-learn
!pip install matplotlib
# !pip install albumentations==1.3.0


In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits import axes_grid1
from IPython.display import clear_output
# import keras_cv
import warnings
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# import keras_cv
import tensorflow as tf
from tensorflow import keras
import albumentations as A
clear_output()
tf.__version__

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
ROOT_DIR_PART_0 = "/kaggle/input/imagenet-1k-tfrecords-ilsvrc2012-part-0"
ROOT_DIR_PART_1 = "/kaggle/input/imagenet-1k-tfrecords-ilsvrc2012-part-1"
train_shard_suffix = 'train-*-of-01024'
val_shard_suffix   = 'validation-*-of-00128'

train_set_path = sorted(
    tf.io.gfile.glob(ROOT_DIR_PART_0 + f'/train/{train_shard_suffix}') +
    tf.io.gfile.glob(ROOT_DIR_PART_1 + f'/train/{train_shard_suffix}')
)
val_set_path = sorted(
    tf.io.gfile.glob(ROOT_DIR_PART_0 + f'/validation/{val_shard_suffix}')
)

len(train_set_path), len(val_set_path)

In [ ]:
batch_size = 16 * strategy.num_replicas_in_sync
image_shape = (224, 224)
IMAGE_SIZE = [224,224]
train_set_len = 626000 + 655167  # 626000 for part 655167 or part 1: 655167
valid_set_len = 50000
train_step = train_set_len // batch_size
val_step = valid_set_len // batch_size
AUTO  = tf.data.AUTOTUNE
TRAINING_FILENAMES = train_set_path
TEST_FILENAMES =  val_set_path
AUG_BATCH =batch_size
BATCH_SIZE =batch_size

In [ ]:
# def deserialization_fn(serialized_example):
#     parsed_example = tf.io.parse_single_example(
#         serialized_example,
#         features={
#             'image/encoded': tf.io.FixedLenFeature([], tf.string),
#             'image/class/label': tf.io.FixedLenFeature([], tf.int64),
#         }
#     )
#     image = tf.image.decode_jpeg(parsed_example['image/encoded'], channels=3)
#     image = tf.image.resize(image, image_shape,method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#     label = tf.cast(parsed_example['image/class/label'], tf.int64) - 1  # [0-999]
#     return image, label

In [ ]:
def cutmix(image, label, PROBABILITY = 1.0):
   
    DIM = IMAGE_SIZE[0]
    CLASSES = 1000
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) 
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2

In [ ]:
def mixup(image, label, PROBABILITY = .5):

    DIM = IMAGE_SIZE[0]
    CLASSES = 1000
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2

In [ ]:
def transform(image,label):
    DIM = IMAGE_SIZE[0]
    CLASSES = 1000
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4

In [ ]:






from tensorflow.keras.layers.experimental import preprocessing

def onehot(image,label):
    CLASSES = 1000
    return image,tf.one_hot(label,CLASSES)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.image.resize(image, IMAGE_SIZE,method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        'image/class/label': tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image/encoded'])
    label = tf.cast(example['image/class/label'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        'image/class/label': tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image/encoded'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    return dataset

# keras_aug = tf.keras.Sequential([
#     preprocessing.RandomFlip("horizontal"),
#     preprocessing.RandomZoom(
#         height_factor=(-0.05, -0.015),
#         width_factor=(-0.05, -0.015)), 
#     preprocessing.RandomContrast(.15),
# #     preprocessing.RandomCrop(120, 120),
    
#     preprocessing.RandomRotation(0.1)
# ])



def album_augment(image):
    transforms = A.Compose([
        
         A.OneOf([
        A.RandomCrop(224, 224),
        A.HorizontalFlip(),
        A.SafeRotate(limit=[-60,60]),
        A.ShiftScaleRotate(p=0.3),
        A.Affine(),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1)),
        A.ToSepia (always_apply=False),
        A.RingingOvershoot (blur_limit=(7, 15), cutoff=(0.7853981633974483, 1.5707963267948966), always_apply=False),
        A.RandomRain(),
        A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=2.5),
        A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0, angle_upper=1, num_flare_circles_lower=6,
                         num_flare_circles_upper=10, src_radius=100, src_color=(255, 255, 255)),     
        A.Cutout(num_holes=2, max_h_size=50, max_w_size=50),
        A.Downscale(scale_min=0.85, scale_max=0.85),
        A.CoarseDropout()],p=.5),
        
        
        A.OneOf([
#         A.Resize (256, 256, interpolation=1, always_apply=True, p=.5),
        A.RandomResizedCrop (180, 180, scale=(0.29, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=1.0),
        A.CenterCrop(224,224),
        A.ColorJitter (brightness=0.25, contrast=0.2, saturation=0.4, hue=0.5, always_apply=False, p=0.5),
        A.RandomCrop(224, 224),
        A.SafeRotate(limit=[-45,45]),
       ],p=.5),

        A.Resize (224, 224, interpolation=1, always_apply=True, p=1)])



    aug_image = transforms(image=image)['image']
    aug_image = tf.cast(x=aug_image, dtype=tf.float32)

# def album_augment(image):
#     transforms = A.Compose([
#              A.OneOf([A.Resize (256, 256, interpolation=1, always_apply=True, p=.5),
#             A.RandomResizedCrop(224, 224)],p=.5),
            
#             A.OneOf([A.HorizontalFlip(),
# #                     A.VerticalFlip(),
#                     A.Rotate (limit=[-45,45]),
# #                     A.Transpose()
#                     ],p=.5),
            
#             A.OneOf([A.ShiftScaleRotate(p=0.3),
#                     A.Affine()],p=.5),
        
#             A.OneOf([A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
#                     A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1)),
#                     A.ToSepia (always_apply=False),
#                     ],p=.5),
        
#             A.OneOf([
#                     A.RingingOvershoot (blur_limit=(7, 15), cutoff=(0.7853981633974483, 1.5707963267948966), always_apply=False),
#                      A.RandomRain(),
#                      A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=2.5),
#                      A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0, angle_upper=1, num_flare_circles_lower=6, num_flare_circles_upper=10, src_radius=100, src_color=(255, 255, 255)),
# #                     A.FancyPCA()
#                     ],p=.5),
# #             A.RandomToneCurve (scale=0.1, always_apply=False, p=0.5),
#             A.OneOf([A.Cutout(num_holes=2, max_h_size=50, max_w_size=50),
#                     A.Downscale(scale_min=0.85, scale_max=0.85),
#                     A.CoarseDropout()],p=.5),
           
#             A.Resize (224, 224, interpolation=1, always_apply=True, p=1)
            
            
#             ], p=1)



#     aug_image = transforms(image=image)['image']
#     aug_image = tf.cast(x=aug_image, dtype=tf.float32)
   
    return aug_image    

def data_augment(image, label):
    aug_func = tf.numpy_function(func=album_augment, inp=[image], Tout=[tf.float32])
    return aug_func,label


def get_training_dataset(dataset, do_aug=True):
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.batch(AUG_BATCH)
    if do_aug: dataset = dataset.map(mixup, num_parallel_calls=AUTO) # note we put AFTER batching
    dataset = dataset.unbatch()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset




def get_validation_dataset(dataset, do_onehot=True):
    dataset = dataset.batch(BATCH_SIZE)
    if do_onehot: dataset = dataset.map(onehot, num_parallel_calls=AUTO) # we must use one hot like augmented train data
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset




In [ ]:
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES))
# augmented_element = all_elements.repeat().batch(AUG_BATCH).map(transform)



In [ ]:
valid = get_validation_dataset(load_dataset(TEST_FILENAMES))

In [ ]:
all_elements

In [ ]:
valid

In [ ]:

row = 6; col = 4;
row = min(row,AUG_BATCH//col)
# all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES)).unbatch()
# augmented_element = all_elements.repeat().batch(AUG_BATCH).map(transform)

for (img,label) in all_elements:
    print()
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:

row = 6; col = 4;
row = min(row,AUG_BATCH//col)
# all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES)).unbatch()
# augmented_element = all_elements.repeat().batch(AUG_BATCH).map(transform)

for (img,label) in valid:
    print()
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:
initial_learning_rate = 0.175
decay_factor = 0.99
decay_epochs = 12.4  # Number of epochs after which to decay the learning rate

# Create a learning rate schedule using a learning rate scheduler
def lr_schedule(epoch):
    lr = initial_learning_rate * (decay_factor ** (epoch / decay_epochs))
    return lr

# Create a learning rate scheduler callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)



In [ ]:
import os
import shutil
os.mkdir("/kaggle/working/imagenet")
os.mkdir("/kaggle/working/imagenet/chief")
files = os.listdir("/kaggle/input/imagenet-nov30/imagenet/chief/")
for i in files:
    shutil.copy("/kaggle/input/imagenet-nov30/imagenet/chief/"+i,"/kaggle/working/imagenet/chief/")

In [ ]:
import tensorflow as tf
import numpy as np
import pickle 
import math
import cv2 
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler
from scipy.ndimage.interpolation import zoom
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam,SGD,Adamax
#from keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D,Add,Concatenate,MaxPooling2D,Activation,GlobalAveragePooling2D,LocallyConnected2D,SeparableConv2D
from tensorflow.keras.layers import Dropout,Cropping2D, Input,DepthwiseConv2D, AveragePooling2D, Flatten, Dense, Reshape, BatchNormalization,Multiply
from tensorflow.keras.layers import UpSampling2D,ZeroPadding2D,Dot,Lambda
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import RandomFourierFeatures
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import albumentations as albu

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
act = "elu"

init = "he_uniform"
# init =  tf.keras.initializers.Ones()
def bock_one(input_data,F1,use_bias, activation, drop_out):

    conv1 = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(input_data)

#     conv1 = BatchNormalization()(conv1)
#     conv1 = Activation(activation)(conv1)

    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv1)
    conv = BatchNormalization()(conv)

#     conv = DepthwiseConv2D((3,3),strides=(1, 1),padding="same",
#                            use_bias=use_bias,
#                            kernel_initializer=init)(conv)
    
    conv = Add()([conv,conv1])
    
    conv2 = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
    
#     conv2 = BatchNormalization()(conv2)
#     conv2 = Activation(activation)(conv2)

    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv2)
    conv = BatchNormalization()(conv)
    
    conv = block_two(conv,use_bias, activation, drop_out)
    # for  _ in range(3):
    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)

    conv = BatchNormalization()(conv)
#     conv = Activation(activation)(conv)
    # conv = BatchNormalization()(conv)
    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
    conv = BatchNormalization()(conv)
    conv = Add()([conv,conv2])
    conv = Activation(activation)(conv)
    
    

    conv4 = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
    
#     conv4 = BatchNormalization()(conv4)
#     conv4 = Activation(activation)(conv4)


    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv4)
    conv = conv = BatchNormalization()(conv)
    
    
#     conv = DepthwiseConv2D((3,3),strides=(1, 1),padding="same",
#                            use_bias=use_bias,
#                            kernel_initializer=init)(conv)
                
    conv = Add()([conv,conv4])
#     conv = Conv2D(F1,(1,1),strides=(1, 1),
#                   padding="same",
#                   use_bias=use_bias,
#                   kernel_initializer="he_uniform")(conv)
    # for _ in range(3):
    conv = Conv2D(F1,(1,1),strides=(1, 1),
                  padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
    
#     conv = BatchNormalization()(conv)
#     conv = Activation(activation)(conv)

    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
                  
#     conv = DepthwiseConv2D((3,3),strides=(1, 1),padding="same",
#                            use_bias=use_bias,
#                            kernel_initializer=init)(conv)
    
    # kernel_regularizer=l2(0.001)
    conv = BatchNormalization()(conv)
    conv = Activation(activation)(conv)
    conv = Dropout(drop_out)(conv)
    conv = Add()([input_data,conv])
    
    #dot  = Dot(axes=1)([input_data,conv])
    #reshape = Reshape((28,28,64))(dot)
    return conv

    


def block_two(input_data,use_bias, activation, drop_out):

    conv = DepthwiseConv2D((5,5),strides=(1, 1),padding="same",
                           use_bias=use_bias,
                           kernel_initializer=init)(input_data)
    
    
    conv = BatchNormalization()(conv)
    conv = Activation(activation)(conv)
#     conv = Dropout(drop_out)(conv)
    F1 = conv.shape[3]
#     conv = ZeroPadding2D((1,1))(conv)
    conv = Conv2D(F1,(1,1),strides=(1, 1),padding="same",
                  use_bias=use_bias,
                  kernel_initializer=init)(conv)
    
    conv = DepthwiseConv2D((3,3),strides=(1, 1),padding="same",
                           use_bias=use_bias,
                           kernel_initializer=init)(conv)

    conv = BatchNormalization()(conv)
    conv = Activation(activation)(conv)
    
    conv = Dropout(drop_out)(conv)
    #conv = Add()([input_data,conv])
    
    return conv

def  dense_layer(input_data,x):
    xx= input_data.shape[1]
    
    reshape = Reshape((1,1,int(xx)))(input_data)
    return reshape

def conv_layer(input_data, F1):
    conv = Conv2D(F1,(1,1),strides=(1, 1),
                  padding="same",
                  use_bias=False,
                  kernel_initializer=init)(input_data)
    conv = BatchNormalization()(conv)
    
    return conv



with strategy.scope():
    input_data =Input((224,224,3))

    conv_1 = conv_layer(input_data,256) #64x64
    conv_2 = bock_one(conv_1,256,False, act, .5)#64x64
    conv_3 = block_two(conv_2,False, act, .5)#64x64
    max_1  = AveragePooling2D()(conv_3)#64x64---->>#16x16

    ################################# 16x16
    conv_5 = bock_one(max_1,256,False, act, .5)#16x16
    conv_6 = conv_layer(conv_5, 256)#16x16

    add_1  = Add()([max_1,conv_6])#  32X32
    conv_7 = conv_layer(add_1, 48)
    conv_77 = bock_one(conv_7,48,False, act, .5)
    conv_77 = block_two(conv_77,False, act, .5)
    # max_2  = MaxPooling2D()(conv_7)
    max_2  = AveragePooling2D()(conv_77)
    #############################
    conv_8 = bock_one(max_2,48,False, act, .5)
    conv_8 = conv_layer(conv_8, 48)
    global_avg_1 = GlobalAveragePooling2D()(conv_8)

    dense_1 =dense_layer(global_avg_1,128)  #128

    #     global_avg_2 = GlobalAveragePooling2D()(dense_1)
    #     reshape_1 = Reshape((1,1,448))(global_avg_2)

    conv_9 = Conv2D(24,(1,1),strides=(1, 1),padding="same",
                      use_bias=False,
                      kernel_initializer=init)(dense_1)
    conv_10 = Conv2D(48,(1,1),strides=(1, 1),padding="same",
                      use_bias=False,
                      kernel_initializer=init)(conv_9)

    add_2= Add()([conv_77,conv_10])

    '''
    **************************************************************************
    '''


    conv_11 = conv_layer(add_2,80) #64x64
    conv_21 = bock_one(conv_11,80,False, act, .5)#64x64
    conv_31 = block_two(conv_21,False, act, .5)#64x64
    max_11  = AveragePooling2D()(conv_31)#64x64---->>#16x16

    conv_51 = bock_one(max_11,80,False, act, .5)#16x16
    conv_61 = conv_layer(conv_51, 80)#16x16

    add_11  = Add()([max_11,conv_61])#  32X32
    conv_71 = conv_layer(add_11, 112)
    conv_771 = bock_one(conv_71,112,False, act, .5)
    conv_771 = block_two(conv_771,False, act, .5)
    # max_2  = MaxPooling2D()(conv_7)
    max_21  = AveragePooling2D()(conv_771)

    conv_81 = bock_one(max_21,112,False, act, .5)
    conv_81 = conv_layer(conv_81, 112)
    global_avg_11 = GlobalAveragePooling2D()(conv_81)

    dense_11 =dense_layer(global_avg_11,128)  #128

    #     global_avg_2 = GlobalAveragePooling2D()(dense_1)
    #     reshape_1 = Reshape((1,1,448))(global_avg_2)

    conv_91 = Conv2D(32,(1,1),strides=(1, 1),padding="same",
                      use_bias=False,
                      kernel_initializer=init)(dense_11)
    conv_101 = Conv2D(112,(1,1),strides=(1, 1),padding="same",
                      use_bias=False,
                      kernel_initializer=init)(conv_91)

    add_21= Add()([conv_771,conv_101])



    '''
    *************************************************************************
    '''



    conv_12 = conv_layer(add_21,224) #64x64
    conv_22 = bock_one(conv_12,224,False, act, .5)#64x64
    conv_32 = block_two(conv_22,False, act, .5)#64x64
    max_12  = AveragePooling2D()(conv_32)#64x64---->>#16x16

    conv_52 = bock_one(max_12,224,False, act, .5)#16x16
    conv_62 = conv_layer(conv_52, 224)#16x16

    add_12  = Add()([max_12,conv_62])#  32X32
    conv_72 = conv_layer(add_12, 512)
    conv_772 = bock_one(conv_72,512,False, act, .5)
    conv_772 = block_two(conv_772,False, act, .5)
# #     max_2  = MaxPooling2D()(conv_7)
#     max_22  = AveragePooling2D()(conv_772)

# #     conv_82 = bock_one(max_22,420,False, act, .5)
# #     conv_82 = conv_layer(conv_82, 420)
#     global_avg_12 = GlobalAveragePooling2D()(max_22)

#     dense_12 =dense_layer(global_avg_12,224)  #128

#     #     global_avg_2 = GlobalAveragePooling2D()(dense_1)
#     #     reshape_1 = Reshape((1,1,448))(global_avg_2)

#     conv_92 = Conv2D(224,(1,1),strides=(1, 1),padding="same",
#                       use_bias=False,
#                       kernel_initializer=init)(dense_12)
#     conv_102 = Conv2D(224,(1,1),strides=(1, 1),padding="same",
#                       use_bias=False,
#                       kernel_initializer=init)(conv_92)

#     add_22= Add()([conv_772,conv_72])



#     '''
#     *************************************************************************
#     '''



#     conv_13 = conv_layer(add_22,380) #64x64
#     conv_23 = bock_one(conv_13,380,False, act, .5)#64x64
#     conv_33 = block_two(conv_23,False, act, .5)#64x64
#     max_13  = AveragePooling2D()(conv_33)#64x64---->>#16x16

#     conv_53 = bock_one(max_13,380,False, act, .5)#16x16
#     conv_63 = conv_layer(conv_53, 380)#16x16

#     add_13  = Add()([max_13,conv_63])#  32X32
#     conv_73 = conv_layer(add_13, 420)
#     conv_773 = bock_one(conv_73,420,False, act, .5)
#     conv_773 = block_two(conv_773,False, act, .5)
#     # max_2  = MaxPooling2D()(conv_7)
#     max_23  = AveragePooling2D()(conv_773)
#     conv_83 = conv_layer(max_23, 480)
#     conv_83 = bock_one(conv_83,480,False, act, .5)
#     conv_83 = conv_layer(conv_83, 480)
   

    conv = GlobalAveragePooling2D()(conv_772)
    conv = Dropout(.75)(conv)
    dense = Dense(1000, activation ="softmax")(conv)
                                                      #categorical_crossentropy
    model = Model(inputs=input_data, outputs=dense)#sparse_categorical_crossentropy,,  Adam(lr=0.0001,decay=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
    model.compile(optimizer=SGD(),#SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True),
                  loss='categorical_crossentropy',  metrics=["accuracy"])
    model.summary()
    
back_up = tf.keras.callbacks.BackupAndRestore(
    "/kaggle/working/imagenet/", save_freq="epoch", delete_checkpoint=True, save_before_preemption=False
)

filename='log.csv'

weight_path="{}_res_unet.h5".format('BEST')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')


history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

history = model.fit( all_elements,
                    validation_data=valid,
                    steps_per_epoch= 1200000 // batch_size,
                          epochs=1000,
                    validation_steps = 25500//batch_size,
                    callbacks=[checkpoint,lr_callback,back_up,history_logger]
#                           validation_data=(X_test,Y_test)
                   )